In [115]:
from sympy import *
import itertools as it
import numpy as np

# Strategic equivalence to weighted zero-sum games

Given a game $u$ find games $z, k$ such that $u = z + k$ with $z$ weighted zero-sum game and $k$ non-strategic game solving non-linear system. For details, see Obsidian note (local file)

`obsidian://open?vault=Work%20diary&file=Notes%2F2024-07-11%20Harmonic%20games%20and%20weighted%20zero%20sum%20games`

# Parameters

In [116]:
# Game skeleton

# numPlayers = 4

# skeleton = np.random.randint(2, 6, numPlayers)

skeleton = [2, 3]
skeleton

[2, 3]

In [117]:
# N
numPlayers = len(skeleton)
players = range(numPlayers)

# -------------------------
# Number of action profiles
# -------------------------

# A
numPures = prod(skeleton)

# List of A_{-i} = for each player, number of action profiles of other players
numPuresMinus = [  int(numPures / skeleton[i]) for i in players ]

# -------------------------
# Number of payoff degrees of freedom
# -------------------------

# AN; number of payoff degrees of freedom
numPays = numPlayers * numPures

In [118]:
# RANDOM PAYOFF
# Payoff in usual Candogan format: flat list of size AN = numPays
# u = np.random.randint(-5, 5, numPays)
# u

In [119]:
# CHOSEN PAYOFF
# Payoff in usual Candogan format: flat list of size AN = numPays

u = [17, -6, -7, 2, -1, -2, -7, 3, 3, 3, -2, -2]

# 223 generalized harmonic
# u = [-4, 12, -46, 34, -26, -49, -4, -1, 0, -2, 0, 3, 99/2, -31, -89, 1, 0, -1, -2, -2, 1, 0, -4, -5, -57/2, -24, 2, -4, -1, -5, 1, 2, -5, 3, -3, -5]

# 2x2 uniform harmonic --> always ~ to exact zero-sum game (as by Candogan's), since players have same # of strategies
# u = [4, -3, 0, 1, -1, 3, 3, -1]
# u  = [0, 4, 1, 3, 0, -1, 1, 2]
# u = [-2, 2, -2, 2, 1, 1, -1, -1]
# u = [-1, 0, -1, 0, -2, -2, 3, 3]

# 3x3 uniform harmonic --> always ~ to exact zero-sum game (as by Candogan's), since players have same # of strategies
# u = [4.66666666666667, 1.66666666666667, -7.33333333333333, 1.33333333333333, 0.333333333333333, -2.66666666666667, 3, -2, -2, -3.00000000000000, -3.00000000000000, 2, 1, -1, -2, 0, 2, -2]
# u = [0.666666666666667, -0.333333333333333, -0.333333333333333, -2.66666666666667, 0.333333333333333, 2.33333333333333, -1, 0, 1, -1.00000000000000, 1.00000000000000, 2, 3, 1, 0, -2, -2, -2]
# u = [2.33333333333333, 6.33333333333333, -0.666666666666667, 4.66666666666667, 0.666666666666667, 2.66666666666667, 3, 3, 2, -4, -8, -3, -3, 1, -3, 3, 3, 2]

# 2x2x2 uniform harmonic --> always ~ to exact zero-sum game (as by Candogan's), since players have same # of strategies
# u = [-2.00000000000000, -5.00000000000000, 1.00000000000000, 9.00000000000000, 2, 0, 1, 0, -1.00000000000000, 10.0000000000000, -3, 3, -2, -2, 2, 3, 2.00000000000000, 0, 2, 0, -2, -2, -3, 1]

In [120]:
assert len(u) == numPays

Number of non-strategic dofs: each player has many dofs as number of action profiles of others; sum over those:

$$\sum_i \prod_{j\neq i}A_j = \sum_i \frac{A}{A_i}$$


In [121]:
# number of non-strategic payoff degrees of freedom
numPaysNS = sum(numPuresMinus)

In [122]:
numPures

6

In [123]:
numPuresMinus

[3, 2]

In [124]:
numPays

12

In [125]:
numPaysNS

5

---
# Program

In [126]:
# Make symbols: methods to dynamically create as many as needed strings and Sympy symbols
# Shift is starting index; default 0

def make_strings(N,s, shift = 0):
	my_strings = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    my_strings.append(tmp_st)
	return my_strings

def make_symbols(N,s, shift = 0):
	my_symbols = []
	for i in range(shift, N + shift):
	    tmp_st = f'{s}{i}'
	    globals()[tmp_st] = Symbol(tmp_st)
	    my_symbols.append(globals()[tmp_st])
	return my_symbols

In [127]:
# Pure actions: list of N lists, each with Ai elements; pure actions of each player

pures_play = [ make_strings(skeleton[i], f'a{i}', shift = 1) for i in players ]

In [128]:
pures_play

[['a01', 'a02'], ['a11', 'a12', 'a13']]

In [129]:
# Pure profiles; cartesian product of pure actions of each player
# Returns one list with A = numPures elements; each element is a tuple of strings

pures = list(it.product(*pures_play))

In [130]:
pures

[('a01', 'a11'),
 ('a01', 'a12'),
 ('a01', 'a13'),
 ('a02', 'a11'),
 ('a02', 'a12'),
 ('a02', 'a13')]

In [131]:
assert len(pures) == numPures

# Payoff of original game

In [132]:
# Pack flat payoff of sizer AN into a list of N lists, each of size A; player-specific payoffs
ui = [ u[ i*numPures : (i+1)*numPures] for i in players]
ui

[[17, -6, -7, 2, -1, -2], [-7, 3, 3, 3, -2, -2]]

In [133]:
# Build payoff dict matching pures with each player-specific payoff
u = [  dict(zip(pures , ui[i] ))  for i in players]

In [134]:
def print_payoff(payoff_dict, payoff_symbol):
    for i in players:
        for a in pures:
            print( f'{payoff_symbol}_{i}{a} = {payoff_dict[i][a]}' )
        print()

In [135]:
# Print payoff
print_payoff(u, 'u')

u_0('a01', 'a11') = 17
u_0('a01', 'a12') = -6
u_0('a01', 'a13') = -7
u_0('a02', 'a11') = 2
u_0('a02', 'a12') = -1
u_0('a02', 'a13') = -2

u_1('a01', 'a11') = -7
u_1('a01', 'a12') = 3
u_1('a01', 'a13') = 3
u_1('a02', 'a11') = 3
u_1('a02', 'a12') = -2
u_1('a02', 'a13') = -2



# Profiles of other players

In [136]:
# Pure profiles of other players
# Make list of N lists; the list pure_minus[i] contains the pure action profiles of players other than i
# Build taking the cartesian product of pure actions of all players other than i
# The size of the list pure_minus[i] is A_{-i} = \prod_{j \neq i} A_j

pures_minus = [ list( it.product( *( pures_play[:i] + pures_play[i+1:] ) ) ) for i in players ]

In [137]:
pures_minus

[[('a11',), ('a12',), ('a13',)], [('a01',), ('a02',)]]

In [138]:
numPuresMinus

[3, 2]

In [139]:
for i in players:
    assert len(pures_minus[i]) == numPuresMinus[i]

# Non-strategic game

In [140]:
# Non-strategic payoff degrees of freedom; will build non-strategic payoff k out of these

# to be determined non-strategic payoff of player i (as many as action profiles of players -i)

n_sym = [make_symbols(numPuresMinus[i], f'alpha{i}', shift = 1) for i in players]
n_sym

[[alpha01, alpha02, alpha03], [alpha11, alpha12]]

In [141]:
# Given the non-strategic dofs, match them with the action profiles of other players

# Build list of N dictionary.
# The keys of the dictionary n_dicts[i] are action profiles a_{-i} of players -i (that is, list of tuples, elements of pures_minus[i] )
# The values of the dictionary n_dicts[i] are the non-strategic payoff of player i given the profile a_{-i}

# read dictionary as: when player -i plays a_{-i} , player i gets n_dicts[i][ a_{-i} ]

n_dicts = [ dict(zip(pures_minus[i], n_sym[i])) for i in players ]

In [142]:
n_dicts

[{('a11',): alpha01, ('a12',): alpha02, ('a13',): alpha03},
 {('a01',): alpha11, ('a02',): alpha12}]

In [143]:
# Util to make (a_i, a_{-i}) given a_i and a_{-i} as tuple of strings (to be used as key for payoff dictionaries)
def make_a(ai, a_minus_i, i):
    l = list(a_minus_i)
    return tuple(l[:i] + [ai] + l[i:])

In [144]:
# Build full non-strategic payoff using non-strategic payoff degrees of freedom
# For each player, k_i does not depend on a_i but only on a_{-i}, so
# for each a_i, for each a_{-i}, build a = (a_i, a_{-i}) and assign k_i(a) = n_dicts[i][ a_{-i} ]

# list of dicts
k = []

for i in players:
    ki = {  }
    for ai in pures_play[i]:
        for a_minus_i in pures_minus[i]:
            a = make_a( ai, a_minus_i, i )
            ki[a] = n_dicts[i][a_minus_i]
    k.append(ki)

# Full payoff of non-strategic game, to be determined
k

[{('a01', 'a11'): alpha01,
  ('a01', 'a12'): alpha02,
  ('a01', 'a13'): alpha03,
  ('a02', 'a11'): alpha01,
  ('a02', 'a12'): alpha02,
  ('a02', 'a13'): alpha03},
 {('a01', 'a11'): alpha11,
  ('a02', 'a11'): alpha12,
  ('a01', 'a12'): alpha11,
  ('a02', 'a12'): alpha12,
  ('a01', 'a13'): alpha11,
  ('a02', 'a13'): alpha12}]

In [145]:
# Check that k has the right size
assert sum( [len(d) for d in k] ) == numPays

# Zero-sum weights

In [146]:
# Coefficients for weighted zero-sum game

lam = make_symbols(numPlayers, 'lambda', shift = 1)
lam

[lambda1, lambda2]

# System

In [147]:
# Unknowns of system to solve: the weights (one per player) + the non-strategic dofs
dofs = lam + list(it.chain(*n_sym))
dofs

[lambda1, lambda2, alpha01, alpha02, alpha03, alpha11, alpha12]

In [148]:
# Number of unknowns of system to solve: the weights (one per player) + the non-strategic dofs
numDofs = numPlayers + numPaysNS
assert len(dofs) == numDofs

## System of non-linear equations

$$\sum_i \lambda_i \,  \big[u_i (\alpha) - k_i(\alpha) \big] = 0 \quad \text{for all } \alpha \in \mathcal{A}$$
- $u_i(\alpha)$ known coefficients
- $\lambda_i$ and $k_i(\alpha)$ unknowns

In [149]:
# --------------------------------------------------------------------------------------
eqs = [  sum( [ lam[i] * ( -u[i][a] + k[i][a] ) for i in players ] )  for a in pures  ]
# --------------------------------------------------------------------------------------

In [150]:
eqs

[lambda1*(alpha01 - 17) + lambda2*(alpha11 + 7),
 lambda1*(alpha02 + 6) + lambda2*(alpha11 - 3),
 lambda1*(alpha03 + 7) + lambda2*(alpha11 - 3),
 lambda1*(alpha01 - 2) + lambda2*(alpha12 - 3),
 lambda1*(alpha02 + 1) + lambda2*(alpha12 + 2),
 lambda1*(alpha03 + 2) + lambda2*(alpha12 + 2)]

In [151]:
sol = nonlinsolve(eqs, dofs)

In [152]:
dofs

[lambda1, lambda2, alpha01, alpha02, alpha03, alpha11, alpha12]

In [153]:
sol

{(0, 0, nan, nan, nan, alpha11, alpha12), (0, 0, nan, nan, alpha03, alpha11, alpha12), (0, 0, nan, alpha02, alpha03, alpha11, alpha12), (0, 0, alpha01, alpha02, alpha03, alpha11, alpha12), (3*lambda2/4, lambda2, -2*(2*alpha12*lambda2 - 9*lambda2)/(3*lambda2), -(4*alpha12*lambda2 + 11*lambda2)/(3*lambda2), -2*(2*alpha12*lambda2 + 7*lambda2)/(3*lambda2), -(-4*alpha12*lambda2 - 5*lambda2)/(4*lambda2), alpha12), (3*lambda2/4, lambda2, -2*(2*alpha12*lambda2 - 9*lambda2)/(3*lambda2), -(4*alpha12*lambda2 + 11*lambda2)/(3*lambda2), -2*(2*alpha12*lambda2 + 7*lambda2)/(3*lambda2), (4*alpha12*lambda2 + 5*lambda2)/(4*lambda2), alpha12), (3*lambda2/4, lambda2, -2*(2*alpha12*lambda2 - 9*lambda2)/(3*lambda2), -(4*alpha12*lambda2 + 11*lambda2)/(3*lambda2), -(4*alpha12*lambda2 + 14*lambda2)/(3*lambda2), -(-4*alpha12*lambda2 - 5*lambda2)/(4*lambda2), alpha12), (3*lambda2/4, lambda2, -2*(2*alpha12*lambda2 - 9*lambda2)/(3*lambda2), -(4*alpha12*lambda2 + 11*lambda2)/(3*lambda2), -(4*alpha12*lambda2 + 14*la

In [154]:
# select sols with lambda1 and lambda2 non-zero (if it exists)
extracted_sol = list(list(sol)[-1])

In [155]:
extracted_sol

[3*lambda2/4,
 lambda2,
 -(4*alpha12*lambda2 - 18*lambda2)/(3*lambda2),
 -(4*alpha12*lambda2 + 11*lambda2)/(3*lambda2),
 -(4*alpha12*lambda2 + 14*lambda2)/(3*lambda2),
 (4*alpha12*lambda2 + 5*lambda2)/(4*lambda2),
 alpha12]

In [156]:
# zip solution in dictionary with dofs
sol_dict = dict(zip(dofs, extracted_sol))

In [157]:
sol_dict

{lambda1: 3*lambda2/4,
 lambda2: lambda2,
 alpha01: -(4*alpha12*lambda2 - 18*lambda2)/(3*lambda2),
 alpha02: -(4*alpha12*lambda2 + 11*lambda2)/(3*lambda2),
 alpha03: -(4*alpha12*lambda2 + 14*lambda2)/(3*lambda2),
 alpha11: (4*alpha12*lambda2 + 5*lambda2)/(4*lambda2),
 alpha12: alpha12}

In [158]:
# fix remaining dofs; to avoid generating zero lambda, generate dictionary with dofs keys and with positive random values
fix_remaining_dofs = dict(zip(dofs, np.random.randint(1, 5, numDofs)))

In [159]:
fix_remaining_dofs

{lambda1: 1,
 lambda2: 2,
 alpha01: 1,
 alpha02: 2,
 alpha03: 4,
 alpha11: 1,
 alpha12: 1}

In [160]:
# fix remaining dofs to random number by subbing in sol_dict
sol_dict_instance = { key : sol_dict[key].subs(fix_remaining_dofs) for key in sol_dict}

In [161]:
sol_dict_instance

{lambda1: 3/2,
 lambda2: 2,
 alpha01: 14/3,
 alpha02: -5,
 alpha03: -6,
 alpha11: 9/4,
 alpha12: 1}

In [162]:
# Sometimes solution returns finite set; convert to float
for key in sol_dict_instance:
    value = sol_dict_instance[key]
    if type(value) == FiniteSet:
        sol_dict_instance[key] = value.args[0]
    elif type(value) == Interval:
        print(f'\nThere is interval solution: {key} in {value}\n')

sol_dict_instance

{lambda1: 3/2,
 lambda2: 2,
 alpha01: 14/3,
 alpha02: -5,
 alpha03: -6,
 alpha11: 9/4,
 alpha12: 1}

In [163]:
# Manually fix solution in interval if needed
#sol_dict_instance[lambda2] = 1
#sol_dict_instance

In [164]:
# util to replace solution dictionary in target dictionary

def replace_in_dict(target_dict, sol_dict):
    return { key : target_dict[key].subs(sol_dict) for key in target_dict }

In [165]:
k_sol = [replace_in_dict(ki, sol_dict_instance) for ki in k]

In [166]:
# Solution : full payoff of non-strategic game
k_sol

[{('a01', 'a11'): 14/3,
  ('a01', 'a12'): -5,
  ('a01', 'a13'): -6,
  ('a02', 'a11'): 14/3,
  ('a02', 'a12'): -5,
  ('a02', 'a13'): -6},
 {('a01', 'a11'): 9/4,
  ('a02', 'a11'): 1,
  ('a01', 'a12'): 9/4,
  ('a02', 'a12'): 1,
  ('a01', 'a13'): 9/4,
  ('a02', 'a13'): 1}]

In [167]:
# Solution: zero-sum weights
lam_sol = [l.subs(sol_dict_instance) for l in lam]
lam_sol

[3/2, 2]

# Build weighted zero sum game
$$u = z+k$$ so $$z = u-k$$

In [168]:
# Weighted zero-sum game full payoff dictionary

z = [ { a: u[i][a] - k_sol[i][a] for a in pures } for i in players   ]

z

[{('a01', 'a11'): 37/3,
  ('a01', 'a12'): -1,
  ('a01', 'a13'): -1,
  ('a02', 'a11'): -8/3,
  ('a02', 'a12'): 4,
  ('a02', 'a13'): 4},
 {('a01', 'a11'): -37/4,
  ('a01', 'a12'): 3/4,
  ('a01', 'a13'): 3/4,
  ('a02', 'a11'): 2,
  ('a02', 'a12'): -3,
  ('a02', 'a13'): -3}]

In [169]:
# Assert is weighted zero-sum
for a in pures:
    weighted_sum = sum( [ lam_sol[i] * z[i][a] for i in players ] )
    assert weighted_sum < 1e-12

# Print results

In [170]:
print(f'\nGame: {skeleton}')
print(f'\nu = z + k')
print('\nu original game')
print('z weighetd zero-sum game')
print('k non-strategic game')
print(f'\nWeights = {lam_sol} such that sum_i λ_i z_i = 0\n' )

sol_exist = True
for l in lam_sol:
    if l == 0:
        sol_exist = False
        print('Solution not found')
        break
if sol_exist: print('Solution found')


Game: [2, 3]

u = z + k

u original game
z weighetd zero-sum game
k non-strategic game

Weights = [3/2, 2] such that sum_i λ_i z_i = 0

Solution found


In [171]:
print_payoff(u, 'u')
print_payoff(z, 'z')
print_payoff(k_sol, 'k')

u_0('a01', 'a11') = 17
u_0('a01', 'a12') = -6
u_0('a01', 'a13') = -7
u_0('a02', 'a11') = 2
u_0('a02', 'a12') = -1
u_0('a02', 'a13') = -2

u_1('a01', 'a11') = -7
u_1('a01', 'a12') = 3
u_1('a01', 'a13') = 3
u_1('a02', 'a11') = 3
u_1('a02', 'a12') = -2
u_1('a02', 'a13') = -2

z_0('a01', 'a11') = 37/3
z_0('a01', 'a12') = -1
z_0('a01', 'a13') = -1
z_0('a02', 'a11') = -8/3
z_0('a02', 'a12') = 4
z_0('a02', 'a13') = 4

z_1('a01', 'a11') = -37/4
z_1('a01', 'a12') = 3/4
z_1('a01', 'a13') = 3/4
z_1('a02', 'a11') = 2
z_1('a02', 'a12') = -3
z_1('a02', 'a13') = -3

k_0('a01', 'a11') = 14/3
k_0('a01', 'a12') = -5
k_0('a01', 'a13') = -6
k_0('a02', 'a11') = 14/3
k_0('a02', 'a12') = -5
k_0('a02', 'a13') = -6

k_1('a01', 'a11') = 9/4
k_1('a01', 'a12') = 9/4
k_1('a01', 'a13') = 9/4
k_1('a02', 'a11') = 1
k_1('a02', 'a12') = 1
k_1('a02', 'a13') = 1



# Findings
Being $\sim$ means being strategically equivalent to a weighted zero-sum game.

## Random
### 2-player
- random $2\times2$ seems always $\sim$
- random $2$-player game other than $2\times2$  seems never $\sim$

### 3-player
- random $3$-player seems often $\sim$

### 4-player
- random $4$-player seems never $\sim$

## Harmonic (generalized, N player)
- **proved: always ~**

# End